### 프로젝트 4
[주제]
교통량 측정

[구현 기능]
1. 영상에 선 긋기
2. 선을 넘는 차량 Detection
3. 선을 넘은 차량 개수 count해서 우측 상단에 표시

In [3]:
# 라이브러리 불러오기
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [4]:
# yolo 모델 불러오기
model = YOLO('yolov8n.pt')
model.info()

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [5]:
# 객체가 선을 넘었는지 확인하는 함수
def check_line_crossing(box, line_y):
    _, y1, _, y2 = map(int, box)
    return y1 <= line_y <= y2

In [21]:
# 영상 불러오기
video = cv2.VideoCapture("cars.mp4")
fps = int(video.get(cv2.CAP_PROP_FPS))
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# mp4 저장용 VideoWriter (루프 밖에서 한 번만 생성)
fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (w, h))

# 객체 카운터 딕셔너리 초기화
object_counts = {}
counted_object_ids = set()  # 이미 카운트된 객체 ID 저장

while True:
    ret, frame = video.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break
    
    line_position = h // 2
    cv2.line(frame, (0, line_position), (w, line_position), (255, 0, 0), 2)

    # YOLO 탐지 + 추적
    results = model.track(frame, persist=True)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            class_name = model.names[cls]

            # tracking ID 가져오기 (없으면 None)
            obj_id = int(box.id[0]) if box.id is not None else None

            # 선 넘었는지 확인
            crossed = check_line_crossing(box.xyxy[0], h//2)

            # 바운딩 박스 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # 카운트 처리: 선을 넘고 아직 카운트 안 된 ID일 경우만
            if crossed and obj_id is not None and obj_id not in counted_object_ids:
                object_counts[class_name] = object_counts.get(class_name, 0) + 1
                counted_object_ids.add(obj_id)  # 이미 센 ID 저장

                # 경고 메시지
                cv2.putText(frame, "Car Crossing",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                            1, (0, 0, 255), 2)

        # 카운트 정보 표시
        y_offset = 50
        for class_name, count in object_counts.items():
            text = f"{class_name}: {count}"
            cv2.putText(frame, text, (w-200, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                        (255, 255, 255), 2)
            y_offset += 30

    # 영상 저장
    out.write(frame)

    # 영상 출력
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

video.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 6 cars, 1 bus, 44.7ms
Speed: 1.7ms preprocess, 44.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 truck, 65.8ms
Speed: 2.3ms preprocess, 65.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 truck, 54.6ms
Speed: 3.4ms preprocess, 54.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 buss, 47.2ms
Speed: 2.1ms preprocess, 47.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 truck, 61.7ms
Speed: 2.3ms preprocess, 61.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 truck, 51.6ms
Speed: 1.9ms preprocess, 51.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 truck, 50.9ms
Speed: 3.4ms preprocess, 50.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 2 buss, 1 tr